In [1]:
%pip install torch
%pip install pymoo
%pip install pywin32

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys

# Get the absolute path of the notebook's directory
notebook_dir = os.getcwd()

# Navigate to the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, ".."))

# Add the parent directory to sys.path so we can import modules
sys.path.append(parent_dir)

# Verify the path
print(f"Added to sys.path: {parent_dir}")

# Add it to sys.path
sys.path.append(parent_dir)

print(parent_dir)
from localityaware.module import *
from NSGA_nn.nsga import optimize_surr_nsga_1, optimize_surr_nsga_2
from FlashOperation.Refrig2DrumHeatExConstr1 import Refrig2DrumConstraintHeatExConstr

Added to sys.path: C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen
['C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen\\NSGA_nn', 'C:\\ProgramData\\anaconda3\\python311.zip', 'C:\\ProgramData\\anaconda3\\DLLs', 'C:\\ProgramData\\anaconda3\\Lib', 'C:\\ProgramData\\anaconda3', '', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen', 'C:\\Users\\flin33\\OneDrive - Johns Hopkins\\ASPEN\\Aspen']


In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import pickle
import os

# Define the file path for saving/loading the data
data_file = "flash_simulation_data_sim_new.pkl"

# Define parameter ranges
flash_1_range_sim = np.linspace(1, 20, 10)
flash_2_range_sim = np.linspace(1, 20, 10)

print("Generating new data...")
assSim = Refrig2DrumConstraintHeatExConstr(AspenFile="../FlashOperation/FlashOperation_HeatExchanger.bkp",
                                        wdpath="../FlashOperation",
                                        visibility=False,
                                        Penalty=1e3)
data_sim = []
for flash_1 in flash_1_range_sim:
    for flash_2 in flash_2_range_sim:
        x_unflat = assSim.unflatten_params([float(flash_1), float(flash_2)])
        data_sim.append([flash_1, flash_2, assSim.run_obj(x_unflat)])

data_sim = np.array(data_sim)

Generating new data...
The current Directory is :  
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen\NSGA_nn
The new Directory where you should also have your Aspen file is : 
C:\Users\flin33\OneDrive - Johns Hopkins\ASPEN\Aspen\FlashOperation
Aspen is active now. If you dont want to see aspen open again take VISIBITLY as False 

TEMPOUT: -65.0630049
The cost is 0.00
TEMPOUT: -65.3737009
The cost is 0.00
TEMPOUT: -63.4166894
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: -67.9285032
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: None
The cost is 0.00
TEMPOUT: -41.5834814
The cost is 0.00
TEMPOUT: -41.3230188
The cost is 0.00
TEMPOUT: -40.1724406
The cost is 0.00
TEMPOUT: -38.9136662
The cost is 0.00
TEMPOUT: -38.9107242
The cost is 0.00
TEMPOUT: -38.9782192
The cost is 0.00
TEMPO

In [5]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata

In [6]:
def plot_contour_surface(data, x_label, y_label, z_label, x_point=None, gradient=None):
    """
    Generate a high-resolution heatmap with contour lines based on the given 3D data.
    Optionally overlays a single point (x_point) and its gradient vector on the plot.
    """
    # Convert data to numpy array for easier manipulation
    data = np.array(data)
    x, y, z = data[:, 0], data[:, 1], data[:, 2]

    # Generate a finer grid for interpolation
    grid_x, grid_y = np.meshgrid(
        np.linspace(x.min(), x.max(), 200),
        np.linspace(y.min(), y.max(), 200)
    )
    
    # Interpolate z values using cubic interpolation
    grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

    # Create the heatmap and add contour lines
    plt.figure(figsize=(8, 6))
    plt.imshow(grid_z, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', aspect='auto')
    plt.colorbar(label=z_label)

    contour_levels = np.linspace(np.nanmin(grid_z), np.nanmax(grid_z), 15)
    plt.contour(grid_x, grid_y, grid_z, levels=contour_levels, colors='black', linewidths=0.8)
    
    # Plot the true optimal point
    true_optimal = np.array([-1.11, -0.5])
    plt.scatter(true_optimal[0], true_optimal[1], color='blue', zorder=5)
    plt.text(true_optimal[0], true_optimal[1], 'True Optimal', color='white', fontsize=10, ha='right')
    
    # Plot the local minimum points
    local_minima = [np.array([0.84, -0.5]), np.array([0.27, -0.5])]
    for lm in local_minima:
        plt.scatter(lm[0], lm[1], color='orange', zorder=5)
        plt.text(lm[0], lm[1], 'Local Min', color='orange', fontsize=10, ha='right')
    
    # Overlay the x point and gradient arrow if provided
    if x_point is not None and gradient is not None:
        # Convert the x and grad lists to NumPy arrays with float type
        gradient = np.asarray(gradient, dtype=float) # grad is grad_pred.tolist() passed in
        plt.quiver(
            x_point[0], x_point[1],
            gradient[0], gradient[1],
            color='red', angles='xy', scale_units='xy', scale=0.1, zorder=10
        )
        plt.scatter(x_point[0], x_point[1], color='red', zorder=10)  # Mark the point
        plt.text(x_point[0], x_point[1], 'x_point', color='red', fontsize=10, ha='left')
    
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

In [ ]:
# Extract columns
x, y, z = data_sim[:, 0], data_sim[:, 1], data_sim[:, 2]

In [ ]:
# Generate a finer grid for interpolation
grid_x, grid_y = np.meshgrid(
    np.linspace(x.min(), x.max(), 200),  # Increased resolution
    np.linspace(y.min(), y.max(), 200)
)

In [ ]:
# Interpolate z values using cubic interpolation
grid_z = griddata((x, y), z, (grid_x, grid_y), method='cubic')

In [ ]:
# # Create a high-resolution heatmap with contour lines
# plt.figure(figsize=(8, 6))
# plt.imshow(grid_z, extent=[x.min(), x.max(), y.min(), y.max()], origin='lower', cmap='viridis', aspect='auto')
# plt.colorbar(label='Objective Value')

In [ ]:
# Add contour lines
# contour_levels = np.linspace(grid_z.min(), grid_z.max(), 15)  # Define contour levels
# plt.contour(grid_x, grid_y, grid_z, levels=contour_levels, colors='black', linewidths=0.8)

In [ ]:
# # Add labels and title
# plt.xlabel('Flash 1')
# plt.ylabel('Flash 2')
# plt.title('Fine-Grained Heatmap with Contour Lines')

In [ ]:
# # Show the plot
# plt.show()

In [7]:
from torch.utils.data import Dataset

class Dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx, :-1]  # all but last column = input
        y = self.data[idx, -1]   # last column = output
        return x, y

    def merge(self, new_data):
        self.data = np.vstack([self.data, new_data])

    def clear(self):
        self.data = np.empty((0, self.data.shape[1]))
        
print(len(data_sim))
print((data_sim))
data_sim = np.array(data_sim)
x_data = data_sim[:, :2]
y_data = data_sim[:, 2]

x_min = x_data.min(axis=0)
x_max = x_data.max(axis=0)


scaler = TorchMinMaxScaler((-1, 1), 
                           max_vals=x_max,
                           min_vals=x_min,
                           min_y=0,
                           max_y=1e6 ,
                           scale_y=True)
#scale data_sim 
data_sim_xscaled, data_sim_yscaled = scaler.transform(data_sim[:, :2], data_sim[:, 2])

#recombine the data
data_sim_scaled = np.column_stack([data_sim_xscaled, data_sim_yscaled])
print(data_sim_scaled)

# **Initialize Model & Datasets**
dataset = Dataset(data_sim_scaled)
model = MLP(2, [20,20,20,20,20], 1)

100
[[ 1.          1.          0.        ]
 [ 1.          3.11111111  0.        ]
 [ 1.          5.22222222  0.        ]
 [ 1.          7.33333333  0.        ]
 [ 1.          9.44444444  0.        ]
 [ 1.         11.55555556  0.        ]
 [ 1.         13.66666667  0.        ]
 [ 1.         15.77777778  0.        ]
 [ 1.         17.88888889  0.        ]
 [ 1.         20.          0.        ]
 [ 3.11111111  1.          0.        ]
 [ 3.11111111  3.11111111  0.        ]
 [ 3.11111111  5.22222222  0.        ]
 [ 3.11111111  7.33333333  0.        ]
 [ 3.11111111  9.44444444  0.        ]
 [ 3.11111111 11.55555556  0.        ]
 [ 3.11111111 13.66666667  0.        ]
 [ 3.11111111 15.77777778  0.        ]
 [ 3.11111111 17.88888889  0.        ]
 [ 3.11111111 20.          0.        ]
 [ 5.22222222  1.          0.        ]
 [ 5.22222222  3.11111111  0.        ]
 [ 5.22222222  5.22222222  0.        ]
 [ 5.22222222  7.33333333  0.        ]
 [ 5.22222222  9.44444444  0.        ]
 [ 5.22222222 11.5555

In [8]:
out = optimize_surr_nsga_2(model=model,
                   old_dataset=dataset,
                   assSim=assSim,
                   lr=1e-4, 
                   epoch=1000,
                   min_vals=scaler.min_x,
                   max_vals=scaler.max_x,
                   scaler=scaler,
                   device='cpu',
                   iter=10,
                   print_loss=True,
                   print_it_data=True,
                   lambda_mse=1,
                   pop_size=100,
                   n_gen = 3,
                   new_data_size=20
                   )

Iteration 0: Training surrogate model...
device cpu
Epoch 0: Total Loss=0.9974
Epoch 10: Total Loss=0.9895
Epoch 20: Total Loss=0.9818
Epoch 30: Total Loss=0.9740
Epoch 40: Total Loss=0.9663
Epoch 50: Total Loss=0.9584
Epoch 60: Total Loss=0.9503
Epoch 70: Total Loss=0.9420
Epoch 80: Total Loss=0.9333
Epoch 90: Total Loss=0.9243
Epoch 100: Total Loss=0.9148
Epoch 110: Total Loss=0.9049
Epoch 120: Total Loss=0.8944
Epoch 130: Total Loss=0.8834
Epoch 140: Total Loss=0.8717
Epoch 150: Total Loss=0.8594
Epoch 160: Total Loss=0.8463
Epoch 170: Total Loss=0.8325
Epoch 180: Total Loss=0.8177
Epoch 190: Total Loss=0.8021
Epoch 200: Total Loss=0.7856
Epoch 210: Total Loss=0.7679
Epoch 220: Total Loss=0.7493
Epoch 230: Total Loss=0.7295
Epoch 240: Total Loss=0.7084
Epoch 250: Total Loss=0.6862
Epoch 260: Total Loss=0.6627
Epoch 270: Total Loss=0.6378
Epoch 280: Total Loss=0.6116
Epoch 290: Total Loss=0.5840
Epoch 300: Total Loss=0.5551
Epoch 310: Total Loss=0.5249
Epoch 320: Total Loss=0.4935
Ep

Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total Loss=0.0000
Epoch 420: Total

Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total 

Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Loss=0.0000
Epoch 390: Total Loss=0.0000
Epoch 400: Total Loss=0.0000
Epoch 410: Total 

Epoch 40: Total Loss=0.0000
Epoch 50: Total Loss=0.0000
Epoch 60: Total Loss=0.0000
Epoch 70: Total Loss=0.0000
Epoch 80: Total Loss=0.0000
Epoch 90: Total Loss=0.0000
Epoch 100: Total Loss=0.0000
Epoch 110: Total Loss=0.0000
Epoch 120: Total Loss=0.0000
Epoch 130: Total Loss=0.0000
Epoch 140: Total Loss=0.0000
Epoch 150: Total Loss=0.0000
Epoch 160: Total Loss=0.0000
Epoch 170: Total Loss=0.0000
Epoch 180: Total Loss=0.0000
Epoch 190: Total Loss=0.0000
Epoch 200: Total Loss=0.0000
Epoch 210: Total Loss=0.0000
Epoch 220: Total Loss=0.0000
Epoch 230: Total Loss=0.0000
Epoch 240: Total Loss=0.0000
Epoch 250: Total Loss=0.0000
Epoch 260: Total Loss=0.0000
Epoch 270: Total Loss=0.0000
Epoch 280: Total Loss=0.0000
Epoch 290: Total Loss=0.0000
Epoch 300: Total Loss=0.0000
Epoch 310: Total Loss=0.0000
Epoch 320: Total Loss=0.0000
Epoch 330: Total Loss=0.0000
Epoch 340: Total Loss=0.0000
Epoch 350: Total Loss=0.0000
Epoch 360: Total Loss=0.0000
Epoch 370: Total Loss=0.0000
Epoch 380: Total Los

In [ ]:
def func(x):
    x1,x2 = x
    # cost_value = (2 - x1)**2 + 100 * (x2 - x1**2)**2
    cost_value = (x1)**2 + (x2)**2
    return cost_value

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(out['y_path'], marker='o', label='Objective Value (y_path)', color='blue')
plt.title('Objective Value Evolution', fontsize=14)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Objective Value', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

print("Objective Value Path (y_path):", out['y_path'])
print("Decision Variable Path (x_path):", out['x_path'])

In [ ]:
# Plot x_path for x1 and x2
x_path_array = np.array(out['x_path'])  # Convert the list of arrays to a NumPy array
plt.figure(figsize=(8, 6))
plt.plot(x_path_array[:, 0], x_path_array[:, 1], marker='o', label='x_path')
plt.title('Path of Decision Variables (x1, x2)', fontsize=14)
plt.xlabel('x1', fontsize=12)
plt.ylabel('x2', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
x = [2,4]
input_tensor = torch.tensor(np.array(x), dtype=torch.float32)
input_tensor_scaled = scaler.transform(input_tensor)
print("input tensor scaled:",input_tensor_scaled)
print("expected out:", func(x))
output_scaled = model(input_tensor_scaled)
input, output = scaler.inverse_transform(input_tensor_scaled, output_scaled)
print("unscaled in out", input,output)


In [ ]:
out['populations']

In [ ]:
import matplotlib.pyplot as plt

def plot_population_evolution(all_pops, min, scaler):
    for i, pop in enumerate(all_pops):
        # Inverse transform the scaled population
        pop = scaler.inverse_transform(pop)
        # Extract x1 and x2 coordinates
        x = pop[:, 0]
        y = pop[:, 1]
        plt.scatter(x, y, alpha=0.5, label=f'Iter {i}')
    
    plt.title("Population Distribution Over Iterations")
    plt.xlabel("x1")
    plt.ylabel("x2")
    #plot the true minimum point
    plt.scatter(min[0], min[1], color='red', marker='*', s=150, label='True Optimal')
    #plot the minimum point found by the model
    plt.scatter(out['x_path'][-1][0], out['x_path'][-1][1], color='green', marker='*', s=150, label='Found Optimal')
    

    plt.legend()
    plt.grid(True)
    plt.show()

plot_population_evolution(out['populations'], [2,4], scaler)


In [ ]:
#plot contour surface of the model
x = np.linspace(-2, 5, 100)
y = np.linspace(-2, 5, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        input_tensor = torch.tensor([X[i, j], Y[i, j]], dtype=torch.float32)
        input_tensor_scaled = scaler.transform(input_tensor)
        output_scaled = model(input_tensor_scaled)
        input, output = scaler.inverse_transform(input_tensor_scaled, output_scaled)
        Z[i, j] = output.item()
#log transform Z
Z = np.log(Z + 1e-10)  # Adding a small constant to avoid log(0)
plt.figure(figsize=(10, 8))
plt.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='Objective Value')
plt.title('Contour Plot of the Model Output')
plt.xlabel('Flash 1')
plt.ylabel('Flash 2')
plt.scatter(out['x_path'][-1][0], out['x_path'][-1][1], color='red', marker='*', s=150, label='Final Point')
#plot the true optimal point
true_optimal = np.array([2, 4])
plt.scatter(true_optimal[0], true_optimal[1], color='blue', zorder=5)
plt.text(true_optimal[0], true_optimal[1], 'True Optimal', color='white', fontsize=10, ha='right')
plt.legend()
plt.show()

In [ ]:
#rosenbrock function
def rosenbrock(x):
    x1, x2 = x
    return (2 - x1)**2 + 100 * (x2 - x1**2)**2

# Generate a grid of points
x = np.linspace(-2, 5, 100)
y = np.linspace(-2, 5, 100)
X, Y = np.meshgrid(x, y)
Z = np.zeros(X.shape)
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = rosenbrock([X[i, j], Y[i, j]])

#log scale the Z values
Z = np.log(Z + 1e-10)  # Adding a small constant to avoid log(0)

plt.figure(figsize=(10, 8))
plt.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='Objective Value log_e')
plt.title('Contour Plot of the Function in Log Scale')
plt.xlabel('Flash 1')
plt.ylabel('Flash 2')
plt.scatter(out['x_path'][-1][0], out['x_path'][-1][1], color='red', marker='*', s=150, label='Final Point')
#plot the true optimal point
true_optimal = np.array([2, 4])
plt.scatter(true_optimal[0], true_optimal[1], color='blue', zorder=5)
plt.text(true_optimal[0], true_optimal[1], 'True Optimal', color='white', fontsize=10, ha='right')
plt.legend()
plt.show()

In [ ]:
x1, x2 = 0.27312183, 0.5264797
result = rosenbrock([x1, x2])
print("Rosenbrock function result:", result)
#compare to 2,4
print("True optimal:", rosenbrock([2, 4]))